# torch.nn.init

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init

In [2]:
# pytorch默认初始化
net = nn.Sequential(
    nn.Linear(4,3),
    nn.ReLU(),
    nn.Linear(3,1)
)
print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [3]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [4]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [5]:
# torch.nn.parameter.Parameter
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

In [6]:
n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [7]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[ 0.2671,  0.0237, -0.3185,  0.4814],
        [ 0.4784,  0.2500, -0.4977,  0.4365],
        [-0.1069,  0.1555, -0.1516,  0.4973]])
None
tensor([[ 0.2680,  0.4094,  0.3406,  0.2661],
        [-0.1596, -0.2437, -0.2028, -0.1584],
        [-0.3760, -0.5743, -0.4779, -0.3733]])


In [8]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 0.0023,  0.0183, -0.0049,  0.0075],
        [ 0.0084, -0.0032,  0.0085, -0.0036],
        [-0.0146, -0.0136, -0.0034, -0.0122]])
2.weight tensor([[ 1.8178e-02, -8.5045e-05, -9.0550e-03]])


## Xavier

In [9]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

In [10]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[ 0.0000, -0.0000,  0.0000, -6.3686],
        [ 0.0000, -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000,  9.2538,  9.9012]])
2.weight tensor([[5.8464, 8.5908, -0.0000]])


In [11]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1.0230, 0.8474, 0.9723])
2.bias tensor([1.3880])


# 共享模型参数
Module类的forward函数里多次调用同一层。

这些共享参数的梯度是累加的。

In [12]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [15]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [16]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
